In [ ]:
# 필요한 라이브러리 설치:
# pip install langchain langgraph openai sentence-transformers pymilvus minio

import os
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Milvus
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph, END
import sqlite3
from minio import Minio
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경변수에서 API_KEY 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")


# 데이터베이스 초기화 (SQLite 예시)
conn = sqlite3.connect('semiconductor.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS parts (
                  id INTEGER PRIMARY KEY AUTOINCREMENT,
                  name TEXT,
                  details TEXT,
                  image_url TEXT)''')
conn.commit()

# MinIO 초기화 (멀티모달 데이터 저장)
minio_client = Minio(
    endpoint="192.168.0.88:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

if not minio_client.bucket_exists("semiconductor-images"):
    minio_client.make_bucket("semiconductor-images")

# 임베딩 및 벡터DB 초기화
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Milvus(
    embedding_function=embedding_model,
    collection_name="semiconductor_parts",
    connection_args={"uri": "http://192.168.0.88:19530"},
    auto_id=True
)


# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o")

# 부품 상세정보 가져오는 함수
def fetch_part_details(part_name):
    response = llm.invoke(f"Provide detailed specifications for the semiconductor part: {part_name}. Include an image URL if available.")
    details = response.content
    image_url = ""  # 실제 이미지 URL 파싱 추가 필요
    return details, image_url

# LangGraph State 정의
from typing import TypedDict

class GraphState(TypedDict):
    part_name: str
    details: str
    image_url: str

# 노드 정의
def query_llm_node(state: GraphState):
    part_name = state["part_name"]
    details, image_url = fetch_part_details(part_name)
    return {"part_name": part_name, "details": details, "image_url": image_url}

def store_data_node(state: GraphState):
    part_name = state["part_name"]
    details = state["details"]
    image_url = state["image_url"]

    # SQLite에 저장
    cursor.execute('INSERT INTO parts (name, details, image_url) VALUES (?, ?, ?)',
                   (part_name, details, image_url))
    conn.commit()

    # (예시) 이미지 MinIO 저장 로직 추가 가능
    # image_data = requests.get(image_url).content
    # minio_client.put_object("semiconductor-images", f"{part_name}.jpg", BytesIO(image_data), len(image_data))

    # 벡터DB 저장
    vector_db.add_texts([details], metadatas=[{"name": part_name}])

    return {"status": "success"}

# LangGraph Workflow 구성
workflow = StateGraph(GraphState)
workflow.add_node("query_llm", query_llm_node)
workflow.add_node("store_data", store_data_node)
workflow.add_edge("query_llm", "store_data")
workflow.set_entry_point("query_llm")
workflow.set_finish_point("store_data")
app = workflow.compile()

# 실행 예시
if __name__ == "__main__":
    part_name = "2N2222 transistor"
    result = app.invoke({"part_name": part_name, "details": "", "image_url": ""})
    print("작업 완료:", result)


작업 완료: {'part_name': '2N2222 transistor', 'details': 'The 2N2222 is a popular NPN bipolar junction transistor (BJT) used for general-purpose low-power amplifying and switching applications. Below are the detailed specifications:\n\n### Electrical Characteristics:\n- **Collector-Emitter Voltage (Vceo)**: 40 V\n- **Collector-Base Voltage (Vcbo)**: 75 V\n- **Emitter-Base Voltage (Vebo)**: 6 V\n- **Collector Current (Ic)**: 800 mA (max)\n- **Base Current (Ib)**: 5 mA (max)\n- **Collector Dissipation (Pc)**: 625 mW\n- **DC Current Gain (hFE)**:\n  - Minimum 100 (at Ic = 10 mA, Vce = 10 V, typical)\n  - Ranges from 100 to 300 typically\n- **Transition Frequency (ft)**: 250 MHz\n- **Noise Figure**: 4 dB (typical)\n\n### Thermal Characteristics:\n- **Junction Temperature (Tj)**: -55°C to +150°C\n- **Storage Temperature Range (Tstg)**: -55°C to +150°C\n\n### Physical and Packaging Information:\n- **Package Type**: TO-18 metal can (also available in TO-92, SOT-23 for different manufacturers)\n- 